# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayora,-0.7393,-90.3518,25.31,90,13,3.13,EC,1688866150
1,1,punta arenas,-53.1500,-70.9167,-1.94,86,0,3.60,CL,1688866151
2,2,qaqortoq,60.7167,-46.0333,6.34,87,41,1.60,GL,1688866151
3,3,tangalle,6.0240,80.7911,27.09,86,100,5.11,LK,1688866152
4,4,isafjordur,66.0755,-23.1240,11.49,65,0,1.70,IS,1688866153
...,...,...,...,...,...,...,...,...,...,...
591,591,aldan,58.6031,125.3894,15.57,48,0,4.59,RU,1688866595
592,592,saint-francois,46.4154,3.9054,19.69,83,100,1.03,FR,1688866596
593,593,san andres,12.5847,-81.7006,29.01,84,40,8.23,CO,1688866596
594,594,north bend,43.4065,-124.2243,20.13,67,20,4.12,US,1688866597


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

map_plot = city_data_df.hvplot(kind='points', x='Lng', y='Lat', c='Humidity', cmap='Blues', colorbar=True, hover_cols=['City', 'Country', 'Humidity'])

# Set plot title and axes labels
map_plot = map_plot.opts(title="Humidity Map", xlabel="Longitude", ylabel="Latitude")

# Display the map plot
map_plot

:Points   [Lng,Lat]   (Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions_df = city_data_df[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) & 
                                   (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]

# Display the filtered DataFrame
print(len(ideal_conditions_df))

19


In [5]:
# Drop any rows with null values
ideal_conditions_df = ideal_conditions_df.dropna()

# Display sample data
ideal_conditions_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
55,55,sparti,37.0733,22.4297,23.86,66,0,1.26,GR,1688866176
74,74,tamanrasset,22.7850,5.5228,26.95,12,0,4.12,DZ,1688866185
81,81,san javier,37.8063,-0.8374,26.99,79,0,1.79,ES,1688866187
99,99,remire-montjoly,4.9167,-52.2667,25.02,88,0,0.00,GF,1688866195
103,103,colonia,50.9333,6.9500,23.87,71,0,0.51,DE,1688866096
114,114,klyuchi,52.2667,79.1667,22.24,52,0,3.44,RU,1688866203
117,117,manzanares,38.9991,-3.3699,24.23,33,0,0.35,ES,1688865928
182,182,brookings,44.3114,-96.7984,21.73,56,0,0.00,US,1688866190
275,275,bastia,42.6667,9.3333,23.89,73,0,2.06,FR,1688866289
297,297,corinto,37.9407,22.9573,24.38,54,0,1.92,GR,1688866300


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_conditions_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = None

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
55,sparti,GR,37.0733,22.4297,66,None
74,tamanrasset,DZ,22.7850,5.5228,12,None
81,san javier,ES,37.8063,-0.8374,79,None
99,remire-montjoly,GF,4.9167,-52.2667,88,None
103,colonia,DE,50.9333,6.9500,71,None
114,klyuchi,RU,52.2667,79.1667,52,None
117,manzanares,ES,38.9991,-3.3699,33,None
182,brookings,US,44.3114,-96.7984,56,None
275,bastia,FR,42.6667,9.3333,73,None
297,corinto,GR,37.9407,22.9573,54,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
radius = 10000  # Specify the desired radius for the hotel search

params = {
    # Add the necessary parameters to the dictionary
    "apiKey": geoapify_key,
    "radius": radius,
    "type": "hotel",
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"category eq 'hotel'"
    params["bias"] = f"proximity({latitude}, {longitude})"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params).json()

    # Convert the API response to JSON format
    if 'features' in response and len(response['features']) > 0:
        hotel_name = response['features'][0]['properties']['name']
        hotel_df.at[index, 'Hotel Name'] = hotel_name
    else:
        hotel_df.at[index, 'Hotel Name'] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
sparti - nearest hotel: No hotel found
tamanrasset - nearest hotel: No hotel found
san javier - nearest hotel: No hotel found
remire-montjoly - nearest hotel: No hotel found
colonia - nearest hotel: No hotel found
klyuchi - nearest hotel: No hotel found
manzanares - nearest hotel: No hotel found
brookings - nearest hotel: No hotel found
bastia - nearest hotel: No hotel found
corinto - nearest hotel: No hotel found
princeville - nearest hotel: No hotel found
damghan - nearest hotel: No hotel found
akbarabad - nearest hotel: No hotel found
la maddalena - nearest hotel: No hotel found
westport - nearest hotel: No hotel found
traben-trarbach - nearest hotel: No hotel found
nova vicosa - nearest hotel: No hotel found
astrakhan - nearest hotel: No hotel found
mohammedia - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
55,sparti,GR,37.0733,22.4297,66,No hotel found
74,tamanrasset,DZ,22.7850,5.5228,12,No hotel found
81,san javier,ES,37.8063,-0.8374,79,No hotel found
99,remire-montjoly,GF,4.9167,-52.2667,88,No hotel found
103,colonia,DE,50.9333,6.9500,71,No hotel found
114,klyuchi,RU,52.2667,79.1667,52,No hotel found
117,manzanares,ES,38.9991,-3.3699,33,No hotel found
182,brookings,US,44.3114,-96.7984,56,No hotel found
275,bastia,FR,42.6667,9.3333,73,No hotel found
297,corinto,GR,37.9407,22.9573,54,No hotel found


In [8]:
# Count how many hotels were found
hotel_counts = hotel_df['Hotel Name'].value_counts()

# Print the count
print(f"Number of hotels found: {len(hotel_counts)}")

Number of hotels found: 1


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot(
    kind='points',
    x='Lng',
    y='Lat',
    c='Humidity',
    cmap='Blues',
    colorbar=True,
    hover_cols=['City', 'Country', 'Humidity'],
    title="Hotel Locations",
    xlabel="Longitude",
    ylabel="Latitude"
)

# Display the map plot
map_plot

:Points   [Lng,Lat]   (Humidity,City,Country)